In [1]:
import torch
import numpy as np
import time

from collections import defaultdict
from typing import List
from conllu import parse_incr, TokenList
from torch import Tensor
from transformers import GPT2Model, GPT2Tokenizer
CUTOFF = None
from lstm.model import RNNModel
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
transformer = GPT2Model.from_pretrained('distilgpt2', output_hidden_states=True)
print("Model ready")
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
print("Tokenizer ready")
# Note that some models don't return the hidden states by default.
# This can be configured by passing `output_hidden_states=True` to the `from_pretrained` method.

Model ready
Tokenizer ready


In [3]:
# The Gulordava LSTM model can be found here: 
# https://drive.google.com/open?id=1w47WsZcZzPyBKDn83cMNd0Hb336e-_Sy
#
# N.B: I have altered the RNNModel code to only output the hidden states that you are interested in.
# If you want to do more experiments with this model you could have a look at the original code here:
# https://github.com/facebookresearch/colorlessgreenRNNs/blob/master/src/language_models/model.py
#
model_location = 'lstm/gulordava.pt'
lstm = RNNModel('LSTM', 50001, 650, 650, 2)
lstm.load_state_dict(torch.load(model_location))


# This LSTM does not use a Tokenizer like the Transformers, but a Vocab dictionary that maps a token to an id.
with open('lstm/vocab.txt') as f:
    w2i = {w.strip(): i for i, w in enumerate(f)}

vocab = defaultdict(lambda: w2i["<unk>"])
vocab.update(w2i)
i2w = { w2i[k]:k for k in w2i}

## Load All Data

In [4]:
from utils import create_or_load_pos_data
from controltasks import save_or_load_pos_controls 
from datasets import find_distribution, POSDataset
import torch.utils.data as data 

train_x, train_y, vocab, words_train = create_or_load_pos_data("train", transformer, tokenizer, cutoff=CUTOFF)
dev_x, dev_y, vocab, words_dev = create_or_load_pos_data("dev", transformer, tokenizer, vocab, cutoff=CUTOFF)
test_x, test_y, vocab, words_test = create_or_load_pos_data("test", transformer, tokenizer, vocab, cutoff=CUTOFF)
dist = find_distribution(data.DataLoader(POSDataset(train_x, train_y), batch_size=1))

flatten_train = [word for sublist in words_train for word in sublist]
flatten_dev   = [word for sublist in words_dev for word in sublist]
flatten_test  = [word for sublist in words_test for word in sublist]

ypos_train_control, ypos_dev_control, ypos_test_control = save_or_load_pos_controls(
    train_x, train_y, [flatten_train, flatten_dev, flatten_test], dist)

Creating data for 12543
Doing LSTM: False
Data created. Pickling now


/home/anna/.local/lib/python3.8/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


Creating data for 2002
Doing LSTM: False
Data created. Pickling now
Creating data for 2077
Doing LSTM: False
Data created. Pickling now


In [5]:
from tree_utils import create_or_load_structural_data
from controltasks import save_or_load_struct_controls

train_xy = create_or_load_structural_data("train", transformer, tokenizer, cutoff=CUTOFF)
dev_xy = create_or_load_structural_data("dev", transformer, tokenizer, cutoff=CUTOFF)
test_xy = create_or_load_structural_data("test", transformer, tokenizer, cutoff=CUTOFF)

struct_train_control, struct_dev_control, struct_test_control = save_or_load_struct_controls(cutoff=CUTOFF)

Fetching for 12543
Doing LSTM: False
Data created,pickling
Fetching for 2002
Doing LSTM: False
Data created,pickling
Fetching for 2077
Doing LSTM: False
Data created,pickling


## PoS Models

In [6]:
# DIAGNOSTIC CLASSIFIER
import torch.nn as nn
class POSProbe(nn.Module):
    def __init__(self, repr_size, pos_size):
        super().__init__()
        self.linear = nn.Linear(repr_size, pos_size)
        
    def forward(self, x):
        return self.linear(x)
    
def eval_given_dataloader(loader, model):
    model.eval()
    correct = 0.0
    total = 0.0
    for x,y in loader:
        x = x.to(device)
        y = y.to(device)
        outputs = model(x)
        preds = torch.argmax(outputs,dim=1)
        c = torch.sum(torch.eq(preds, y))
        correct += c.item()
        total += y.shape[0]
    return correct/total
    
def train(my_model, train_loader, dev_loader, epoch_amount = 10):
    ce = nn.CrossEntropyLoss()
    optim = torch.optim.Adam(my_model.parameters())
    for i in range(epoch_amount):
        my_model.train()
        epoch_correct = 0.0
        epoch_total = 0.0
        for x,y in train_loader:
            
            x = x.to(device)
            y = y.to(device)
            outputs = my_model(x)
            preds = torch.argmax(outputs,dim=1)
            correct = torch.sum(torch.eq(preds, y))
            accuracy = correct.item()/y.shape[0]
            loss = ce(outputs, y)

            optim.zero_grad()
            loss.backward()
            optim.step()
            
            epoch_correct += correct.item()
            epoch_total += y.shape[0]
        print("Epoch",i,"accuracy", epoch_correct/epoch_total, eval_given_dataloader(dev_loader, my_model))

In [7]:
# Normal task
ntrain_loader = data.DataLoader(POSDataset(train_x, train_y), batch_size=16)
ndev_loader = data.DataLoader(POSDataset(dev_x, dev_y), batch_size=16)
ntest_loader = data.DataLoader(POSDataset(test_x, test_y), batch_size=16)

model = POSProbe(768, len(dist)).to(device)
train(model, ntrain_loader, ndev_loader, 10)
print("Test accuracy", eval_given_dataloader(ntest_loader, model))

Epoch 0 accuracy 0.8959356746584549 0.8773262287259425
Epoch 1 accuracy 0.9208641884791162 0.8822172737394623
Epoch 2 accuracy 0.9263240218002298 0.8846429139494194
Epoch 3 accuracy 0.9286262433707261 0.8903292508350564
Epoch 4 accuracy 0.9299802038272601 0.891561953236838
Epoch 5 accuracy 0.9310066720434049 0.8911643073007793
Epoch 6 accuracy 0.9317887430652296 0.8911643073007793
Epoch 7 accuracy 0.9324632793215534 0.8893351359949101
Epoch 8 accuracy 0.9329471857663074 0.8910847781135677
Epoch 9 accuracy 0.9333724368844246 0.8889772546524575
Test accuracy 0.8864759324195091


In [8]:
# Normal task
ctrain_loader = data.DataLoader(POSDataset(train_x, ypos_train_control), batch_size=16)
cdev_loader = data.DataLoader(POSDataset(dev_x, ypos_dev_control), batch_size=16)
ctest_loader = data.DataLoader(POSDataset(test_x, ypos_test_control), batch_size=16)
print(len(ypos_train_control), len(train_x))
print(len(ypos_dev_control), len(dev_x))
print(len(ypos_test_control), len(test_x))

model = POSProbe(768, len(dist)).to(device)
train(model, ctrain_loader, cdev_loader, 10)
print("Test accuracy", eval_given_dataloader(ctest_loader, model))

204585 204585
25148 25148
25096 25096
Epoch 0 accuracy 0.5241097832196886 0.5309368538253539
Epoch 1 accuracy 0.5565852824009581 0.5376173055511373
Epoch 2 accuracy 0.559801549478212 0.5392078892953714
Epoch 3 accuracy 0.5609355524598577 0.5386909495784953
Epoch 4 accuracy 0.5615465454456583 0.5390090663273421
Epoch 5 accuracy 0.5619326930126842 0.5390885955145538
Epoch 6 accuracy 0.5620548916098443 0.5390885955145538
Epoch 7 accuracy 0.5622064178703229 0.5392476538889772
Epoch 8 accuracy 0.5623090646919373 0.5390885955145538
Epoch 9 accuracy 0.5623921597380062 0.5391283601081597
Test accuracy 0.5321166719795983


## Structural

In [9]:
import torch.nn as nn
import torch


class StructuralProbe(nn.Module):
    """ Computes squared L2 distance after projection by a matrix.
    For a batch of sentences, computes all n^2 pairs of distances
    for each sentence in the batch.
    """
    def __init__(self, model_dim, rank, device="cpu"):
        super().__init__()
        self.probe_rank = rank
        self.model_dim = model_dim
        
        self.proj = nn.Parameter(data = torch.zeros(self.model_dim, self.probe_rank))
        
        nn.init.uniform_(self.proj, -0.05, 0.05)
        self.to(device)

    def forward(self, batch):
        """ Computes all n^2 pairs of distances after projection
        for each sentence in a batch.
        Note that due to padding, some distances will be non-zero for pads.
        Computes (B(h_i-h_j))^T(B(h_i-h_j)) for all i,j
        Args:
          batch: a batch of word representations of the shape
            (batch_size, max_seq_len, representation_dim)
        Returns:
          A tensor of distances of shape (batch_size, max_seq_len, max_seq_len)
        """
        transformed = torch.matmul(batch, self.proj)
        
        batchlen, seqlen, rank = transformed.size()
        
        transformed = transformed.unsqueeze(2)
        transformed = transformed.expand(-1, -1, seqlen, -1)
        transposed = transformed.transpose(1,2)
        
        diffs = transformed - transposed
        
        squared_diffs = diffs.pow(2)
        squared_distances = torch.sum(squared_diffs, -1)

        return squared_distances

    
class L1DistanceLoss(nn.Module):
    """Custom L1 loss for distance matrices."""
    def __init__(self):
        super().__init__()

    def forward(self, predictions, label_batch, length_batch):
        """ Computes L1 loss on distance matrices.
        Ignores all entries where label_batch=-1
        Normalizes first within sentences (by dividing by the square of the sentence length)
        and then across the batch.
        Args:
          predictions: A pytorch batch of predicted distances
          label_batch: A pytorch batch of true distances
          length_batch: A pytorch batch of sentence lengths
        Returns:
          A tuple of:
            batch_loss: average loss in the batch
            total_sents: number of sentences in the batch
        """
        labels_1s = (label_batch != -1).float()
        predictions_masked = predictions * labels_1s
        labels_masked = label_batch * labels_1s
        total_sents = torch.sum((length_batch != 0)).float()
        squared_lengths = length_batch.pow(2).float()

        if total_sents > 0:
            loss_per_sent = torch.sum(torch.abs(predictions_masked - labels_masked), dim=(1,2))
            normalized_loss_per_sent = loss_per_sent / squared_lengths
            batch_loss = torch.sum(normalized_loss_per_sent) / total_sents
        
        else:
            batch_loss = torch.tensor(0.0)
        
        return batch_loss, total_sents

In [10]:
from torch import optim
import math
import tree_utils

# I recommend you to write a method that can evaluate the UUAS & loss score for the dev (& test) corpus.
# Feel free to alter the signature of this method.
def evaluate_probe(probe, dataloader):
    loss_function =  L1DistanceLoss()
    probe.eval()
    total_loss = 0.0
    total_uuas = 0.0
    amt = 0.0
    for distances, embs, lengths in dataloader:
        embs = embs.to(device)
        distances = distances.to(device)
        lengths = lengths.to(device)
        amt += len(distances)
        outputs = probe(embs)
        loss = loss_function(outputs, distances, lengths)[0]
        total_loss += loss.item()
        for i in range(len(distances)):
            l = lengths[i]
            preds = outputs[i,0:l, 0:l]
            gold = distances[i,0:l, 0:l]
            
            u = tree_utils.calc_uuas(preds, gold)
            if math.isnan(u):
                amt -= 1
            # This if statement is a hack so nans don't get counted
            if u >= 0: total_uuas += u
    
    return total_loss/amt, total_uuas/amt

# Feel free to alter the signature of this method.
def train_structural(probe, dataloader, dev_dataloader,test_loader, epochs=100):
    lr = 1e-5
    batch_size = 128
    
    optimizer = optim.Adam(probe.parameters(), lr=lr)
    #scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5,patience=1)
    loss_function =  L1DistanceLoss()

    for epoch in range(epochs):
        probe.train()
        for distances, embs, lengths in dataloader:
            embs = embs.to(device)
            distances = distances.to(device)
            lengths = lengths.to(device)
            outputs = probe(embs)
            loss = loss_function(outputs, distances, lengths)[0]
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        dev_loss, dev_uuas = evaluate_probe(probe, dev_dataloader)
        print("Epoch", epoch, "Dev loss and uuas", dev_loss, dev_uuas)
        # Using a scheduler is up to you, and might require some hyper param fine-tuning
        #scheduler.step(dev_loss)

    test_loss, test_uuas = evaluate_probe(probe, test_loader)
    print("Test loss, uuas", test_loss, test_uuas)


In [13]:

from datasets import StructuralDataset, pad_batch

batch_size = 32
train_loader = data.DataLoader(StructuralDataset(*train_xy), batch_size=batch_size, collate_fn= pad_batch)
dev_loader = data.DataLoader(StructuralDataset(*dev_xy), batch_size=batch_size, collate_fn= pad_batch)
test_loader = data.DataLoader(StructuralDataset(*test_xy), batch_size=batch_size, collate_fn= pad_batch)

emb_dim = 768
rank = 64
probe = StructuralProbe(emb_dim, rank).to(device)
print(probe)
train_structural(probe, train_loader, dev_loader, test_loader, epochs=100)

StructuralProbe()
Epoch 0 Dev loss and uuas 2.5447451674725103 0.2562101074791884
Epoch 1 Dev loss and uuas 1.589373900436578 0.26002308270152175
Epoch 2 Dev loss and uuas 1.044382552619236 0.2652228065912948
Epoch 3 Dev loss and uuas 0.7241506009949243 0.2752651475166392
Epoch 4 Dev loss and uuas 0.5286934759338572 0.2936645898767807
Epoch 5 Dev loss and uuas 0.40306736868889426 0.31899045225358164
Epoch 6 Dev loss and uuas 0.31722051738815227 0.3485226062280033
Epoch 7 Dev loss and uuas 0.25490789308156875 0.3677320221766217
Epoch 8 Dev loss and uuas 0.2072790497610371 0.3771351016868157
Epoch 9 Dev loss and uuas 0.169515687835204 0.38076268915636563
Epoch 10 Dev loss and uuas 0.1390038615897627 0.38418137647964984
Epoch 11 Dev loss and uuas 0.11424544231121973 0.3894909850757561
Epoch 12 Dev loss and uuas 0.09432310592990317 0.39363481505158915
Epoch 13 Dev loss and uuas 0.07853567405955397 0.39782617989280683
Epoch 14 Dev loss and uuas 0.06622627010856894 0.40232968404554936
Epoch 

KeyboardInterrupt: 

In [ ]:
from datasets import StructuralDataset, pad_batch

batch_size = 32
ctrain_loader = data.DataLoader(StructuralDataset(struct_train_control, train_xy[1]), batch_size=batch_size, collate_fn= pad_batch)
cdev_loader = data.DataLoader(StructuralDataset(struct_dev_control, dev_xy[1]), batch_size=batch_size, collate_fn= pad_batch)
ctest_loader = data.DataLoader(StructuralDataset(struct_test_control, test_xy[1]), batch_size=batch_size, collate_fn= pad_batch)

emb_dim = 768
rank = 64
probe = StructuralProbe(emb_dim, rank).to(device)
print(probe)
train_structural(probe, ctrain_loader, cdev_loader, ctest_loader, epochs=100)